In [1]:
import networkx as nx
import csv
import numpy as np
from random import randint
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

In [2]:
# Create a graph
G = nx.read_edgelist('edgelist.txt', delimiter=',', create_using=nx.DiGraph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

Number of nodes: 138499
Number of edges: 1091955


In [3]:
# Read the abstract of each paper
abstracts = dict()
with open('abstracts.txt', 'r',encoding='utf-8') as f:
    for line in f:
        node, abstract = line.split('|--|')
        abstracts[int(node)] = abstract

In [4]:
# Map text to set of terms
for node in abstracts:
    abstracts[node] = set(abstracts[node].split())

In [5]:
# Read the authors of each paper
authors = dict()
with open('authors.txt', 'r',encoding='utf-8') as f:
    for line in f:
        node, author = line.split('|--|')
        authors[int(node)] = author 

In [6]:
# Map text to set of terms
for node in authors:
    authors[node] = set(authors[node].split(','))

In [7]:
# its class label is 1 if it corresponds to an edge and 0, otherwise.
# Use the following 3 features for each pair of nodes:
# (1) sum of number of unique terms of the two nodes' abstracts
# (2) absolute value of difference of number of unique terms of the two nodes' abstracts
# (3) number of common terms between the abstracts of the two nodes
# (4) number of common authors
# (5) The node degree is the number of edges adjacent to the node. 
#     The weighted node degree is the sum of the edge weights for edges incident to that node.
# (6) absolute value of difference of the weighted node degree 
# (7) The node in_degree is the number of edges pointing to the node. 
# (8) absolute value of difference of the number od edges pointing to the node
# (9) sum of degree centrality of the two nodes

X_train = np.zeros((2*m, 9))
y_train = np.zeros(2*m)
n = G.number_of_nodes()
DEGREE_CENTRALITY = nx.degree_centrality(G)
# CLOSENESS_CENTRALITY = nx.closeness_centrality(G)
# BETWEENNESS_CENTRALITY = networkx.betweenness_centrality(G)
for i,edge in enumerate(G.edges()):
    # an edge
    X_train[i,0] = len(abstracts[edge[0]]) + len(abstracts[edge[1]])
    X_train[i,1] = abs(len(abstracts[edge[0]]) - len(abstracts[edge[1]]))
    X_train[i,2] = len(abstracts[edge[0]].intersection(abstracts[edge[1]]))
    
    X_train[i,3] = len(authors[edge[0]].intersection(authors[edge[1]]))
    
    X_train[i,4] = G.degree(edge[0]) + G.degree(edge[1]) 
    X_train[i,5] = abs(G.degree(edge[0]) - G.degree(edge[1]))
    
    X_train[i,6] = G.in_degree(edge[0]) + G.in_degree(edge[1])
    X_train[i,7] = abs(G.in_degree(edge[0]) - G.in_degree(edge[1]))
    
    X_train[i,8] = DEGREE_CENTRALITY[edge[0]] + DEGREE_CENTRALITY[edge[1]]
    
    y_train[i] = 1

    # a randomly generated pair of nodes
    n1 = randint(0, n-1)
    n2 = randint(0, n-1)
    
    #an edge
    X_train[m+i,0] = len(abstracts[n1]) + len(abstracts[n2])
    X_train[m+i,1] = abs(len(abstracts[n1]) - len(abstracts[n2]))
    X_train[m+i,2] = len(abstracts[n1].intersection(abstracts[n2]))
    
    X_train[m+i,3] = len(authors[n1].intersection(authors[n2]))
    
    X_train[m+i,4] = G.degree(n1) + G.degree(n2) 
    X_train[m+i,5] = abs(G.degree(n1) - G.degree(n2))
    
    X_train[m+i,6] = G.in_degree(n1) + G.in_degree(n2)
    X_train[m+i,7] = abs(G.in_degree(n1) - G.in_degree(n2))
    
    X_train[m+i,8] = DEGREE_CENTRALITY[n1] + DEGREE_CENTRALITY[n2]
    
    y_train[m+i] = 0

In [8]:
print('Size of training matrix:', X_train.shape)

Size of training matrix: (2183910, 9)


In [9]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))

In [10]:
# Create the test matrix. Use the same 8 features as above
X_test = np.zeros((len(node_pairs), 9))
for i,node_pair in enumerate(node_pairs):
    X_test[i,0] = len(abstracts[node_pair[0]]) + len(abstracts[node_pair[1]])
    X_test[i,1] = abs(len(abstracts[node_pair[0]]) - len(abstracts[node_pair[1]]))
    X_test[i,2] = len(abstracts[node_pair[0]].intersection(abstracts[node_pair[1]]))
    X_test[i,3] = len(authors[node_pair[0]].intersection(authors[node_pair[1]]))
    X_test[i,4] = G.degree(node_pair[0]) + G.degree(node_pair[1])
    X_test[i,5] = abs(G.degree(node_pair[0]) - G.degree(node_pair[1]))
    X_test[i,6] = G.in_degree(node_pair[0]) + G.in_degree(node_pair[1])
    X_test[i,7] = abs(G.in_degree(node_pair[0]) - G.in_degree(node_pair[1]))
    X_test[i,8] = DEGREE_CENTRALITY[node_pair[0]] + DEGREE_CENTRALITY[node_pair[1]]

print('Size of test matrix:', X_test.shape)

Size of test matrix: (106692, 9)


In [11]:
X_train, y_train = shuffle(X_train, y_train)

In [12]:
# Use logistic regression to predict if two nodes are linked by an edge
clf = LogisticRegression(solver='liblinear',random_state=34)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
y_pred = y_pred[:,1]

In [22]:
# Use logistic regression to predict if two nodes are linked by an edge
# clf = LogisticRegression(solver='saga', C=1, n_jobs=-1,random_state=34)
# clf.fit(X_train, y_train)
# y_pred = clf.predict_proba(X_test)
# y_pred = y_pred[:,1]

C:\Users\Johnny\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [25]:
#  from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(max_depth=3, random_state=42)
# clf.fit(X_train, y_train)
# y_pred = clf.predict_proba(X_test)
# y_pred = y_pred[:,1]

In [13]:
# Write predictions to a file
predictions = zip(range(len(y_pred)), y_pred)
with open("submission_text_graph_link_prediction.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row)